<a href="https://colab.research.google.com/github/Mark65537/MugenSGDKConv/blob/main/mugen_sgdk_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mugen SGDK converter
Essa ferramenta foi criada para apartir de imagens estraidas do mugen e o arquivo .air gerar imagens compativeis com animação de sprites da SGDK.

## Proximos adições


*   ~Arquivo com as hitbox de cada movimento~
*   ~Arquivo com o frameData de cada movimento~
*   Arquivo com os gatilhos para cada movimento
*   ~Arquivo res com os movimentos capturados~

## Instruções de uso
  1. Carregar o arquivo air
  2. Carregar as imagens em uma pasta escolhida
  3. Preencher as variaveis de entrada
  4. Clicar no menu superio Ambiente de execução -> Executar tudo
  5. Quando o script for concluido (pode levar alguns minutos) aparecerá um arquivo output.zip para download.



## Variaveis de entrada
modifique as variaveis para cada sprites


*   **pasta_das_imagens** = pasta onde vão estar as imagens extraidas do arquivo .sff
*   **prefixo** = prefixo da imagens ex: '**Krauzer**_0-0.png' Krauzer é o prefixo
*   **arquivo_air** = path completo para o arquivo air ex: /content/krauser.air
*   **save_to_sgdk** = se marcado salva saida com as cores do megadrive e no modo indexado, caso contrario salva um png com cores não indexadas
*   **hamoopig** = saem os arquivos com padrão hamoopig

In [ ]:
#@title Variaveis de entrada
pasta_das_imagens = "/content/duck/" #@param {type:"string"}
prefixo = "duck" #@param {type:"string"}
arquivo_air= "/content/duck/Duck_King.air" #@param {type:"string"}
save_to_sgdk = True #@param {type:"boolean"}
hamoopig = False #@param {type:"boolean"}

x_scale = 1 #@param {type:"slider", min:0, max:1, step:0.01}
y_scale = 1 #@param {type:"slider", min:0, max:1, step:0.01}

In [ ]:
import cv2
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import os
import numpy as np
from PIL import Image
import copy
import re
import math


In [ ]:
class move():
  def __init__(self, id):
    self.id = id
    self.frames = []

  def add_frame(self, frame):
    self.frames.append(frame)

class hitbox():
  def __init__(self, type, x1, y1, x2,y2):
    self.type = type
    self.x1 = int(x1)
    self.y1 = int(y1)
    self.x2 = int(x2)
    self.y2 = int(y2)

class sprite_frame():
  def __init__(self, group, number, x_offset, y_offset,frames, hitboxs, flipV, flipH):
    self.group = group
    self.number = number
    self.x_offset = int(x_offset)
    self.y_offset = int(y_offset)
    self.frames = frames
    self.hitboxs = hitboxs
    self.flipV = flipV
    self.flipH = flipH

  def add_hitbox(self, hitbox):
    self.hitboxs.append(hitbox)


In [ ]:
def convert_tiles ( value):
  if value % 8 != 0:
    return value + 8 - value % 8
  else:
    return value

def img_to_gen(img):
  img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  for i in range(len(img2)):
    for j in range(len(img2[i])):
      if len(img2[i][j]) == 4:
        img2[i][j] = img2[i][j] & 0xe0e0e0ff
      else:
        img2[i][j] = img2[i][j] & 0xe0e0e0

  img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2BGR)
  return img2

def convert_1d_to_2d(l, cols):
    return [l[i:i + cols] for i in range(0, len(l), cols)]

def convert_palette(img):
  img = img.quantize()
  unique = convert_1d_to_2d(img.getpalette(),3)
  buffer = copy.copy(unique[0])
  k = 0
  for i in range(len(unique)):
    if np.all(unique[i] == [0xe0, 0x0, 0xe0]):
      unique[i] = buffer
      k = k + 1
  if k > 0:
    unique[0] = [0xe0, 0x0, 0xe0]

  unique = [j for sub in unique for j in sub]
  return unique

def save_8bpp(img, name, alpha):
  if hamoopig:
    if not os.path.exists('/content/hamoopig/'):
      os.makedirs('/content/hamoopig/')
    img_path = '/content/hamoopig/' + str(name) + '.png'
  else:
    if not os.path.exists('/content/output/'):
      os.makedirs('/content/output/')
    img_path = '/content/output/' + str(name) + '.png'

  # img = img_to_gen(img)
  if alpha:
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)
  else:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = Image.fromarray(img)
  palette = convert_palette(img)
  if 768 % len(palette) != 0:
    raise Exception('error', 'invalid palette size')

  scale = 768//len(palette)
  if alpha:
    img1 = img.convert(mode='RGB')
  else:
    img1 = img.convert(mode='RGB')

  p_img = Image.new('P', (16, 16))
  p_img.putpalette(palette*scale)
  conv = img1.quantize(palette=p_img, dither=0)
  conv.save(img_path)
  return conv

def save_result(img, name):
  if not os.path.exists('/content/output/'):
    os.makedirs('/content/output/')
  cv2.imwrite('/content/output/' + str(name) + '.png', img)

In [ ]:
def generate_res(name, size_x, size_y):
  line = 'SPRITE spr_'+name+' "sprite/' + name+'_8bpp.png" ' + str(size_x) + ' ' + str(size_y) + ' FAST 0\n'

  if not os.path.exists('/content/output/res'):
    os.makedirs('/content/output/res')
  res_path = '/content/output/res/' + str(name) + '.res'
  with open(res_path, 'w') as file:
    file.write(line)


def generate_code( frames, name, id, hitboxs, x_offset, y_offset, lenx, leny, file = None, file2 = None):
  hibox_lines = []
  hibox_lines2 = []
  i =0
  j = 0
  line = ""
  line2 =""
  # for hitboxList in hitboxs:
  #   for hitbox in hitboxList:
  #     x = min(hitbox.x1, hitbox.x2) + lenx//2 + x_offset[i]
  #     w = abs(hitbox.x1 - hitbox.x2)
  #     y = min(hitbox.y1, hitbox.y2) + leny -1 + y_offset[i]
  #     h = abs(hitbox.y1 - hitbox.y2)
  #     if hitbox.type == 'atack':
  #       box = '_hitbox'
  #     else:
  #       box = '_hurtbox'
  #     hibox_lines.append('const box ' + str(name) + box + id + '_{}_{}'.format(i,j) + ' = {{.x= {}, .w= {}, .y={}, .h= {}}};\n'.format(x,w,y,h))
  #     hibox_lines2.append('const box ' + str(name) + box + id + '_{}_{}'.format(i,j) + ';\n')

  #     j = j+1
  #   i = i+1
  #   j = 0:
  if len(frames) > 0:
    line = 'const s16 ' + str(name) + '_frameData_'+id +'[' + str(len(frames))+ '] = {' + ', '.join(map(str, frames)) + '};\n'
    line2 = 'const s16 ' + str(name) + '_frameData_'+id +'[' + str(len(frames))+ '];\n'
  if file == None:
    if not os.path.exists('/content/output/code'):
      os.makedirs('/content/output/code')
    res_path = '/content/output/code/' + str(name) + '_frame_data.c'
    file = open(res_path, 'w')
    file.write('#include "'+ str(name) + '_frame_data.h"\n\n')

  if file2 == None:
    if not os.path.exists('/content/output/code'):
      os.makedirs('/content/output/code')
    res_path = '/content/output/code/' + str(name) + '_frame_data.h'
    file2 = open(res_path, 'w')
    define = '__'+ str(name + '_frame_Data').upper() + '__'
    file2.write('#ifndef '+ define +'\n#define '+define + '\n\n')

  file.write(line)
  file2.write(line2)
  for l in hibox_lines:
    file.write(l)

  for l in hibox_lines2:
    file2.write(l)
  return file, file2

In [ ]:
mugen_ids = [0, 5, 6, 10, 11, 12, 20, 21, 40, 41, 42, 47, 120, 122, 130, 132, 140, 142, 150, 151, 181, 190, 191, 192, 195, 200, 210, 230, 240, 400, 410, 430, 440, 5002, 5005, 5006, 5012, 5015, 5016, 5090, 5100, 5160]
hpig_ids = [607, 608, 601, 200, 602, 420, 410, 606, 300, 320, 310, 610, 207, 107, 208, 108, 209, 501, 502, 503, 611, 101, 102, 104, 105, 201, 202, 204, 205, 301, 302, 304, 305, 506, 507, 508, 511, 512, 513, 551, 570, 552]
def mugen_id_to_hpig(id):
  if int(id) in mugen_ids:
    return str(hpig_ids[mugen_ids.index(int(id))])
  return None


In [ ]:
def draw_hitbox(img, hitboxs):
  for hitbox in hitboxs:
    x1 = hitbox.x1 + img.shape[1]//2
    x2 = hitbox.x2 + img.shape[1]//2
    y1 = hitbox.y1 + img.shape[0] -1
    y2 = hitbox.y2 +  img.shape[0]-1

    # print(x1, x2, y1, y2, img.shape)

    # print(hitbox.y1,  hitbox.y2)
    if hitbox.type == 'atack':
      color = (0,0,255, 255)
    else:
      color = (255,0,0, 255)
    img = cv2.rectangle(img, (x1, y1), (x2, y2), color, 1)
  return img
# draw_hitbox(imgs[0],moves[0].frames[0].hitboxs[0] )
# id = '2'
# frames = ['2', '4', '-2', '5']

In [ ]:
fase = 'init'
fase_before = fase
moves = []
hitboxs = []
i = -1
with open(arquivo_air, encoding="cp437", errors='ignore') as air_file:
  for line in air_file:
    if 'Begin' in line:
      if fase == 'init':
        i = i+1
      else:
        moves.pop()
      fase_before = fase
      fase = 'read_id'
    if fase=='read_id':
      l = re.findall(r'-?\d+', line)
      if(l):
        id = l[0]
      if hamoopig:
        if mugen_id_to_hpig(id) is not None:
          moves.append(move(mugen_id_to_hpig(id)))
        else:
          i = i-1
      else:
        moves.append(move(id))
      fase_before = fase
      fase = 'hit_box_n'
      continue
    if fase== 'hit_box_n':
      hitboxs = []
      if 'Clsn2' in line:
        hitbox_type = 'defence'
        # print(line.split(), id)
        hitbox_n = int(line.split(':')[1])
        fase_before = fase
        fase = 'hitbox_read'
        continue
      if 'Clsn1' in line:
        hitbox_type = 'atack'
        # print(line.split())
        hitbox_n = int(line.split(':')[1])
        fase_before = fase
        fase = 'hitbox_read'
        continue
      else:
        fase_before = fase
        fase = 'read_frame'
    if fase=='hitbox_read':
      l = re.findall(r'-?\d+', line)
      if hitbox_n > 0:
        hitboxs.append(hitbox(hitbox_type, *l[2:6]))
        hitbox_n = hitbox_n -1
      else:
        fase_before = fase
        fase = 'read_frame'
    if fase == 'read_frame':
      if len(line.split(',')) >= 5:
        l = re.findall(r'-?\d+', line)
        if 'H' in line:
          flipH = True
        else:
          flipH = False
        if 'V' in line:
          flipV = True
        else:
          flipV = False

        f = sprite_frame(*l[0:5], hitboxs, flipH, flipV)
        moves[i].add_frame(f)
      elif 'Clsn2' in line:
        hitboxs = []
        hitbox_type = 'defence'
        hitbox_n = int(line.split(':')[1])
        fase_before = fase
        fase = 'hitbox_read'
        # print('defence')
        continue
      elif 'Clsn1' in line:
        hitbox_type = 'atack'
        # print(line.split())
        hitbox_n = int(line.split(':')[1])
        fase_before = fase
        fase = 'hitbox_read'
        # print('atack')
        continue
      elif line == '\n':
        fase_before = fase
        fase = 'init'
      else:
        fase_before = fase
        fase = 'read_frame'


In [ ]:
file = None
file2 = None
file3 = None
alpha = False
i = 0
imgs = []
xmax = 0
ymax = 0
xmin = 1000
ymin = 1000
frame_max = 0
for m in moves:
  imgs.append([])
  frame_data = []
  boxs = []
  fail = False
  for frame in m.frames:
    image_path = pasta_das_imagens +'/' + prefixo + '_'+ frame.group +'-' + frame.number +'.png'
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if img is not None:
      width = int(img.shape[1] * x_scale)
      height = int(img.shape[0]* y_scale )
      dim = (width, height)
      img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
      frame_data.append(frame.frames)
      boxs.append(frame.hitboxs)
      imgs[i].append(img)
      #resize ofsets
      frame.x_offset = math.ceil(frame.x_offset * x_scale)
      frame.y_offset = math.ceil(frame.y_offset * y_scale)

      if img.shape[1] + frame.x_offset > xmax:
        xmax = img.shape[1] + frame.x_offset
      if img.shape[0] + frame.y_offset > ymax:
        ymax = img.shape[0] +  frame.y_offset
      if frame.x_offset < xmin:
        xmin = frame.x_offset
      if frame.y_offset < ymin:
        ymin = frame.y_offset
      if frame.y_offset != 0:
        pass
        # print(frame.x_offset, frame.y_offset, m.id)
      # print(img.shape)
    else:
      # moves.pop(moves.index(m))
      # imgs.pop()
      # i = i - 1
      print("ids fail", m.id)
      fail = True
      break

  if len(m.frames) > frame_max and not fail:
    frame_max = len(m.frames)
  i = i+1
  m.frame_data = frame_data


if imgs:
    background_color = imgs[0][0][0][0]
    alpha = True
valid_frames = 0
for j in range(len(moves)):
    if len(imgs[j]) > 0:
        valid_frames = valid_frames+1


if imgs:
  xmax_frame = convert_tiles(math.ceil(xmax-xmin))
  xmax = xmax_frame*frame_max
  ymax_frame = convert_tiles(math.ceil(ymax - ymin))
  ymax = ymax_frame*valid_frames
  # print('max values', xmax_frame, ymax_frame, xmax)

  final = np.zeros((ymax,xmax,len(background_color)), np.uint8)
  # final_hbox = np.zeros((ymax,xmax,len(background_color)), np.uint8)
  final[:]= background_color
  # final_hbox[:] = background_color

  k = 1
  for j in range(len(moves)):
    # print('offsets')
    xoffs = []
    yoffs = []

    for i in range(len(imgs[j])):
      try:
        # x0ff = moves[j].frames[i].x_offset -xmin + i*xmax_frame
        x0ff = moves[j].frames[i].x_offset - int(imgs[j][i].shape[1]/2) + int(xmax_frame/2)  + i*xmax_frame
        xoffs.append(moves[j].frames[i].x_offset)
        y0ff = ymax_frame*(k) - imgs[j][i].shape[0] - (moves[j].frames[i].y_offset -ymin)
        yoffs.append(y0ff)

        if moves[j].frames[i].flipV:
          if moves[j].frames[i].flipH:
            imgs[j][i] = cv2.flip(imgs[j][i], -1)
          else:
            imgs[j][i] = cv2.flip(imgs[j][i], 1)
        elif moves[j].frames[i].flipH:
          imgs[j][i] = cv2.flip(imgs[j][i], 0)
        try:
          final[y0ff:y0ff+imgs[j][i].shape[0], x0ff:x0ff+imgs[j][i].shape[1]] = imgs[j][i]
          # img2 = draw_hitbox(imgs[i],m.frames[i].hitboxs )
          # final_hbox[y0ff:y0ff+img2.shape[0], x0ff:x0ff+img2.shape[1]] = img2
        except:
          print('falha ao criar movimento: ', moves[j].id, i)
          print(x0ff, y0ff,moves[j].frames[i].x_offset, moves[j].frames[i].y_offset, xmax_frame, ymax, xmin, ymin)
      except:
         print('falha ao adicionar frame: ', moves[j].id, i)
         continue
    if len(imgs[j]) > 0:
      k = k+1
    try:
      file2, file3 = generate_code(moves[j].frame_data, prefixo, moves[j].id, boxs,xoffs, yoffs,xmax_frame, ymax_frame,  file2, file3)
    except:
      print('falha ao adicionar criar arquivo framedata: ', moves[j].id, i)
if save_to_sgdk:
  save_8bpp(final, prefixo +'_8bpp', alpha)
else:
  save_result(final,  prefixo +'_32bpp')
generate_res(prefixo, int(xmax_frame/8), int(ymax_frame/8))
# cv2_imshow(final)
print(valid_frames)
# file.close()
if file2:
  file2.close()
if file3:
  file3.write('\n#endif')
  file3.close()
  # display(save_8bpp(final, 'Krauzer_'+ move.id + '_8bpp'))

In [ ]:
if hamoopig:
  !zip -r /content/hamoopig.zip /content/hamoopig/
else:
  !zip -r /content/output.zip /content/output/
